<a href="https://colab.research.google.com/github/jeffblackadar/sensation/blob/main/sensory_map_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Package installation, Mount Google Drive

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install packages
!apt-get install libgeos-dev
!pip install https://github.com/matplotlib/basemap/archive/master.zip
!pip install geopandas
!pip install contextily

# 2. Relocate map coordinates to get ready to unroll map at your location's latitude longitude
* Clicking a location in Google Maps will provide this.

In [93]:
# Input the location to unroll the map at.
local_lat = 45.475871
local_lon = -75.549315

#local_lat = 45.477661
#local_lon = -75.536143

In [94]:
shapefiles_path = "/content/drive/MyDrive/github/sensation/shapefiles/"
from geographiclib.geodesic import Geodesic
import geopy
from geopy.distance import VincentyDistance
import geopandas as gpd
from shapely.geometry import Point, Polygon
import os

def transition_point(origin_lat, origin_lon, dest_lat, dest_lon, new_map_origin_lat, new_map_origin_lon):
    #print(origin_lat, origin_lon, dest_lat, dest_lon, new_map_origin_lat, new_map_origin_lon)  
    geod = Geodesic.WGS84.Inverse(origin_lat, origin_lon, dest_lat, dest_lon)
    distance = geod['s12']
    bearing = geod['azi1']
    origin = geopy.Point(new_map_origin_lat, new_map_origin_lon)
    destination = VincentyDistance(meters=distance).destination(origin, bearing)
    lat2, lon2 = destination.latitude, destination.longitude
    return(lat2, lon2)


geotif_crs = 4326
new_map_shp = gpd.GeoDataFrame()
new_map_shp['geometry'] = None
new_map_shp.crs = ("EPSG:" + str(geotif_crs))
new_map_shp.geometry = new_map_shp.geometry.to_crs(crs=geotif_crs)
new_map_shp.to_crs(crs=geotif_crs)
new_map_shp = new_map_shp.to_crs(epsg=geotif_crs)

print("#--------------------------------------------------------")
print("# Copy this below, paste in the cell in section 3 of this notebook to provide instructions to generate the new map")
print("#--------------------------------------------------------")

h_df = gpd.read_file(os.path.join(shapefiles_path,"hopewell_selected_elements.shp"))
for index, row in h_df.iterrows():
    #print(row['type'], row['name'], row['geometry'])
    p = row['geometry']
    lat2, lon2 = transition_point(40.20595294149071, -75.77220799054444, p.y, p.x, local_lat, local_lon)

    point = Point(lon2, lat2)
    new_p_row = {'id':row['id'], 'geometry':point, 'type':row['type'], 'name':row['name']}
    t_name = str(row['name'])
    if(t_name.lower()=='none'):
        t_name=""
    t_desc = str(row['desc'])
    if(t_desc.lower()=='none'):
        t_desc=""
    print('gm.genMarker('+str(lat2)+', '+str(lon2)+',"'+str(row['type'])+'","'+t_name+'","'+t_desc+'")')
    new_map_shp = new_map_shp.append(new_p_row, ignore_index=True)

new_map_shp.to_file(os.path.join(shapefiles_path,"new_map.shp"))


new_map_shp_poly = gpd.GeoDataFrame()
new_map_shp_poly['geometry'] = None
new_map_shp_poly.crs = ("EPSG:" + str(geotif_crs))
new_map_shp_poly.geometry = new_map_shp_poly.geometry.to_crs(crs=geotif_crs)
new_map_shp_poly.to_crs(crs=geotif_crs)
new_map_shp_poly = new_map_shp_poly.to_crs(epsg=geotif_crs)
polygon_points_array = []

h_df = gpd.read_file(os.path.join(shapefiles_path,"hopewell_selected_elements_polys.shp"))
for index, row in h_df.iterrows():
    poly = row['geometry']
    polygon_points = ""
    comma = ""
    for p in poly.exterior.coords:
        lat2, lon2 = transition_point(40.20595294149071, -75.77220799054444, p[1], p[0], local_lat, local_lon)
        polygon_points = polygon_points + comma + "["+str(lat2)+","+str(lon2)+"]"
        comma = ","
        #print(lat2, lon2)
    #/content/drive/MyDrive/crane_obs/crane_hopewell/hopewell_selected_elements.shp
        point = Point(lon2, lat2)
        polygon_points_array.append(point)
    polygon = Polygon(polygon_points_array)
    new_p_row = {'id':row['id'], 'geometry':polygon, 'type':row['type'], 'name':row['name']}
    print("gm.genPolygon('["+polygon_points+"]','"+str(row['type'])+"')")
    new_map_shp_poly = new_map_shp_poly.append(new_p_row, ignore_index=True)

new_map_shp_poly.to_file(os.path.join(shapefiles_path,"new_map_polys.shp"))

#test marker
origin = geopy.Point(local_lat, local_lon)
destination = VincentyDistance(meters=2000).destination(origin, 270)
lat2, lon2 = destination.latitude, destination.longitude
print('gm.genTestMarker('+str(lat2)+', '+str(lon2)+')')
print("")
print("#--------------------------------------------------------")
print("# Copy this above, paste in the cell in section 3 of this notebook to provide instructions to generate the new map")
print("#--------------------------------------------------------")

#--------------------------------------------------------
# Copy this below, paste in the cell in section 3 of this notebook to provide instructions to generate the new map
#--------------------------------------------------------
gm.genMarker(45.47890524246566, -75.56611403321713,"RCH","RCH","")
gm.genMarker(45.478045775348, -75.56241738538371,"RCH","","")
gm.genMarker(45.479084619032285, -75.5606251644505,"RCH","","")
gm.genMarker(45.479801606521505, -75.55691209024718,"RCH","","")
gm.genMarker(45.478952924359234, -75.55542219985323,"RCH","","")
gm.genMarker(45.4793838133057, -75.55354605510716,"RCH","","")
gm.genMarker(45.479780549057914, -75.5525676092125,"RCH","","")
gm.genMarker(45.470290416814905, -75.55228904428883,"RCH","","")
gm.genMarker(45.47033550770272, -75.55056360470792,"RCH","","")
gm.genMarker(45.46974425824116, -75.55151372730964,"RCH","","")
gm.genMarker(45.468732503626555, -75.55196028059133,"RCH","","")
gm.genMarker(45.4692836321735, -75.55304573662096,"RCH","",""

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import geopandas as gpd
import pandas as pd
import contextily as ctx

print("Original Map")
h_df = gpd.read_file(os.path.join(shapefiles_path,"hopewell_selected_elements.shp"))

#print(h_df.crs)
# Change crs to one compatible with basemap
h_df = h_df.to_crs(epsg=3857) #3857

ax = h_df.plot(figsize=(20, 20), alpha=0.5, edgecolor='k')
ctx.add_basemap(ax, zoom=15)

h_df['coords'] = h_df['geometry'].apply(lambda x: x.representative_point().coords[:])
h_df['coords'] = [coords[0] for coords in h_df['coords']]
for idx, row in h_df.iterrows():
    plt.annotate(s=row['type'], xy=row['coords'], horizontalalignment='center')

print("Unrolled map at your location")
new_map_df = gpd.read_file(os.path.join(shapefiles_path,"new_map.shp"))

print(new_map_df.crs)
# Change crs to one compatible with basemap
new_map_df = new_map_df.to_crs(epsg=3857) #3857
print(new_map_df.crs)

ax = new_map_df.plot(figsize=(20, 20), alpha=0.5, edgecolor='k')
ctx.add_basemap(ax, zoom=13)

new_map_df['coords'] = new_map_df['geometry'].apply(lambda x: x.representative_point().coords[:])
new_map_df['coords'] = [coords[0] for coords in new_map_df['coords']]
for idx, row in new_map_df.iterrows():
    plt.annotate(s=row['type'], xy=row['coords'], horizontalalignment='center')

new_map_df = gpd.read_file(os.path.join(shapefiles_path,"new_map_polys.shp"))
#new_map_df.boundary.plot(ax=ax,color="red",alpha=0.3)

# 3. Generate the map to display at your location

In [87]:
import json
class GenerateLeafletMap:
    def __init__(self,html_file_name):
        self.html_file_name = html_file_name
        self.html_file = open(self.html_file_name, 'w') 
        self.markers = 0
        self.polygons = 0
        self.polylines = 0
        self.layers = 0
        self.type_data = {}
    def close(self):
        self.html_file.close()

    def set_type_data_dict(self,type_data_dict):
        self.type_data = type_data_dict

    def genMapHTMLTop(self):
        self.html_file.write('<!DOCTYPE html>\n')
        self.html_file.write('<html>\n')
        self.html_file.write('<head><meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n')
        self.html_file.write('<title>map title</title>\n')
        self.html_file.write('<link href="https://cdnjs.cloudflare.com/ajax/libs/leaflet/0.7.5/leaflet.css" rel="stylesheet" />\n')
        self.html_file.write('<script src="https://cdnjs.cloudflare.com/ajax/libs/leaflet/0.7.5/leaflet.js"></script>\n')
        self.html_file.write('<script src="https://code.jquery.com/jquery-2.1.4.min.js"></script>\n')
        self.html_file.write('<style type="text/css">#my-map {\n')
        self.html_file.write('  width:960px;\n')
        self.html_file.write('  height:700px;\n')
        self.html_file.write('}\n')
        self.html_file.write('h1 {\n')
        self.html_file.write('  color: #AA0000;\n')
        self.html_file.write('  font-size: 24px;\n')
        self.html_file.write('  font-family: verdana;\n')
        self.html_file.write('}\n')
        self.html_file.write('h2 {\n')
        self.html_file.write('  color: #000000;\n')
        self.html_file.write('  font-size: 20px;\n')
        self.html_file.write('  font-family: verdana;\n')
        self.html_file.write('}\n')
        self.html_file.write('h3 {\n')
        self.html_file.write('  color: #000000;\n')
        self.html_file.write('  font-size: 16px;\n')
        self.html_file.write('  font-family: verdana;\n')
        self.html_file.write('}\n')    
        self.html_file.write('p {\n')
        self.html_file.write('  color: #996600;\n')
        self.html_file.write('  font-family: verdana;\n')
        self.html_file.write('}\n')
        self.html_file.write('li {\n')
        self.html_file.write('  color: #996600;\n')
        self.html_file.write('  font-family: verdana;\n')
        self.html_file.write('}\n')  
        self.html_file.write('th {\n')
        self.html_file.write('  text-align: right;\n')
        self.html_file.write('  font-family: verdana;\n')
        self.html_file.write('  color: #663300;\n')
        self.html_file.write('}\n')
        self.html_file.write('td {\n')
        self.html_file.write('  text-align: right;\n')
        self.html_file.write('  font-family: verdana;\n')
        self.html_file.write('  color: #663300;\n')
        self.html_file.write('}\n')
        self.html_file.write('</style>\n')
        self.html_file.write('</head>\n')
        self.html_file.write('<body>\n')

    def genMapHTMLH1(self,map_title):
        self.html_file.write('<h1>'+map_title+'</h1>\n')
        self.html_file.write('<div id="description">description</div>\n')
        self.html_file.write('<div id="latitude">latitude</div>\n')
        self.html_file.write('<div id="longitude">longitude</div>\n')
        self.html_file.write('<div id="description_point"></div>\n')
        self.html_file.write('<div id="description_poly"></div>\n')
                    
    def genMapHTMLScriptTop(self): 
        self.html_file.write('  <div id="my-map"></div>\n')
        self.html_file.write('    <script>\n')
        self.html_file.write('    // This file was generated by this R program:  https://github.com/jeffblackadar/oc-transpo-maps/blob/master/oc_4_gen_html.r\n')
        self.html_file.write('var map;\n')
        self.html_file.write("map = L.map('my-map');\n")
        self.html_file.write('var markers = [];\n')
        self.html_file.write('var polygons = [];\n')
        self.html_file.write('var markers_data = [];\n')
        self.html_file.write('var polygons_data = [];\n')

        self.html_file.write("// Thank you:\n")
        self.html_file.write("// https://stackoverflow.com/questions/27928/calculate-distance-between-two-latitude-longitude-points-haversine-formula\n")
        self.html_file.write("function distance(lat1, lon1, lat2, lon2) {\n")
        self.html_file.write("  var p = 0.017453292519943295;    // Math.PI / 180\n")  
        self.html_file.write("  var c = Math.cos;\n")
        self.html_file.write("  var a = 0.5 - c((lat2 - lat1) * p)/2 + \n")
        self.html_file.write("          c(lat1 * p) * c(lat2 * p) * \n")
        self.html_file.write("          (1 - c((lon2 - lon1) * p))/2;\n")
        self.html_file.write("  return 12742 * Math.asin(Math.sqrt(a)); // 2 * R; R = 6371 km\n")
        self.html_file.write("}\n")

        self.html_file.write("//https://stackoverflow.com/questions/31790344/determine-if-a-point-reside-inside-a-leaflet-polygon\n")
        self.html_file.write("function isMarkerInsidePolygon(marker, poly) {\n")
        self.html_file.write("    var polyPoints = poly.getLatLngs();       \n")
        self.html_file.write("    var x = marker.getLatLng().lat, y = marker.getLatLng().lng;\n")
        self.html_file.write("    var inside = false;\n")
        self.html_file.write("    for (var i = 0, j = polyPoints.length - 1; i < polyPoints.length; j = i++) {\n")
        self.html_file.write("        var xi = polyPoints[i].lat, yi = polyPoints[i].lng;\n")
        self.html_file.write("        var xj = polyPoints[j].lat, yj = polyPoints[j].lng;\n")
        self.html_file.write("        var intersect = ((yi > y) != (yj > y)) && (x < (xj - xi) * (y - yi) / (yj - yi) + xi);\n")
        self.html_file.write("        if (intersect) inside = !inside;\n")
        self.html_file.write("    }\n")
        self.html_file.write("    return inside;\n")
        self.html_file.write("};\n")
        self.html_file.write("function update_per_proximity(marker) {\n")
        self.html_file.write("    var marker_lat = marker.getLatLng().lat, marker_lon = marker.getLatLng().lng;\n")

        self.html_file.write("// Polygons - check if marker is inside.  If so, light it up.\n")  
        self.html_file.write("    for (cn=0;cn<polygons.length;cn++){\n")
        self.html_file.write("        if(isMarkerInsidePolygon(marker, polygons[cn])==true){\n")
        self.html_file.write("            polygons[cn].setStyle({fillOpacity: 0.7, weight:2,opacity: 0.3});\n")
        self.html_file.write('            var popup_text_poly = polygons_data[cn].sight + " " + polygons_data[cn].hearing + " " + polygons_data[cn].feeling + " " + polygons_data[cn].smell;\n')        
        self.html_file.write("            document.getElementById('description_poly').innerHTML = popup_text_poly;\n")
        self.html_file.write("            polygons[cn]._popup.setContent(popup_text_poly);\n")
        self.html_file.write("        }\n")
        self.html_file.write("    }\n")
#    "hill":'{"color":"brown","sight":"hilly terrain","hearing":"wind blowing through trees","smell":"damp leaves","feeling":"cool, damp"}',
        self.html_file.write("    for (cn=0;cn<markers.length;cn++){\n")
        self.html_file.write("        dist_to = distance(marker_lat, marker_lon,markers[cn].getLatLng().lat,markers[cn].getLatLng().lng)\n")
        self.html_file.write("        // adjust distance for proximity sensitivity\n")
        self.html_file.write("        if(dist_to < 0.1){\n")

        self.html_file.write("            if(markers_data[cn].type=='rch'){\n")
        self.html_file.write("                markers[cn].setIcon(iconRCH);\n")
        
        self.html_file.write("                //markers[cn]._popup.setContent('RCH  <audio controls><source src=\"sounds/rch.wav\" type=\"audio/wav\"> Your browser does not support the audio element. </audio>')\n")
        self.html_file.write("            }\n")
        self.html_file.write("\n")

        self.html_file.write("            if(markers_data[cn].type=='furnace'){\n")
        self.html_file.write("                markers[cn].setIcon(iconFurnace);\n")
        self.html_file.write("                //markers[cn]._popup.setContent('Furnace  <audio controls><source src=\"sounds/furnace.wav\" type=\"audio/wav\"> Your browser does not support the audio element. </audio>')\n")
        self.html_file.write("            }\n")
        self.html_file.write("\n")

        self.html_file.write("            if(markers_data[cn].type=='business'){\n")
        self.html_file.write("                markers[cn].setIcon(iconBusiness);\n")
        self.html_file.write("                //markers[cn]._popup.setContent('Business  <audio controls><source src=\"sounds/business.wav\" type=\"audio/wav\"> Your browser does not support the audio element. </audio>')\n")
        self.html_file.write("            }\n")
        self.html_file.write("\n")

        self.html_file.write("            if(markers_data[cn].type=='house'){\n")
        self.html_file.write("                markers[cn].setIcon(iconHouse);\n")
        self.html_file.write("                //markers[cn]._popup.setContent('House  <audio controls><source src=\"sounds/house.wav\" type=\"audio/wav\"> Your browser does not support the audio element. </audio>')\n")
        self.html_file.write("            }\n")
        self.html_file.write("\n")    
        self.html_file.write('            var desc_temp = markers_data[cn].desc;')
        self.html_file.write('            if(desc_temp.trim().length>0){desc_temp = "This is described on the map as: "+desc_temp+". "};')
        self.html_file.write('            var popup_text = markers_data[cn].sight + " " + markers_data[cn].hearing + " " + markers_data[cn].feeling + " " + markers_data[cn].smell + " " + desc_temp;\n')
        self.html_file.write("            document.getElementById('description_point').innerHTML = popup_text;\n")
        self.html_file.write("            var audio_temp = markers_data[cn].sound;\n")
        self.html_file.write("            if(audio_temp.trim().length>0){audio_temp = '<audio controls><source src=\"sounds/'+audio_temp+'\"> Your browser does not support the audio element. </audio>'}")
        self.html_file.write("            markers[cn]._popup.setContent(popup_text + audio_temp)\n")


        self.html_file.write("        } // if dist_to\n")
        self.html_file.write("    } // for\n")

        self.html_file.write("} // function\n")                      
      
        self.html_file.write('var iconHouse = L.icon({\n')
        self.html_file.write('    iconUrl: "icons/house.png",\n')
        self.html_file.write('    //shadowUrl: "leaf-shadow.png",\n')

        self.html_file.write('    iconSize:     [30, 37], // size of the icon\n')
        self.html_file.write('    //shadowSize:   [50, 64], // size of the shadow\n')
        self.html_file.write('    iconAnchor:   [15, 36], // point of the icon which will correspond to markers location\n')
        self.html_file.write('    //shadowAnchor: [4, 62],  // the same for the shadow\n')
        self.html_file.write('    popupAnchor:  [-3, -38] // point from which the popup should open relative to the iconAnchor\n')
        self.html_file.write('});\n')

        self.html_file.write('var iconRCH = L.icon({\n')
        self.html_file.write('    iconUrl: "icons/rch.png",\n')
        self.html_file.write('    iconSize:     [35, 33], // size of the icon\n')
        self.html_file.write('    iconAnchor:   [17, 32], // point of the icon which will correspond to \n')
        self.html_file.write('    popupAnchor:  [-3, -34] // point from which the popup should open relative to the iconAnchor\n')
        self.html_file.write('});\n')
      
        self.html_file.write('var iconFurnace = L.icon({\n')
        self.html_file.write('    iconUrl: "icons/furnace.png",\n')
        self.html_file.write('    iconSize:     [34, 38], // size of the icon\n')
        self.html_file.write('    iconAnchor:   [17, 37], // point of the icon which will correspond to \n')
        self.html_file.write('    popupAnchor:  [-3, -39] // point from which the popup should open relative to the iconAnchor\n')
        self.html_file.write('});\n')

        self.html_file.write('\n')
        self.html_file.write('var iconBusiness = L.icon({\n')
        self.html_file.write('    iconUrl: "icons/business.png",\n')
        self.html_file.write('    iconSize:     [32, 32],\n')
        self.html_file.write('    iconAnchor:   [16, 32],\n')
        self.html_file.write('    popupAnchor:  [-3, -33]\n')
        self.html_file.write('});\n')

        self.html_file.write('\n')
        self.html_file.write('var iconTravel = L.icon({\n')
        self.html_file.write('    iconUrl: "icons/travel.png",\n')
        self.html_file.write('    iconSize:     [32, 32],\n')
        self.html_file.write('    iconAnchor:   [16, 32],\n')
        self.html_file.write('    popupAnchor:  [-3, -33]\n')
        self.html_file.write('});\n')

        self.html_file.write('\n')
        self.html_file.write('var iconComputer = L.icon({\n')
        self.html_file.write('    iconUrl: "icons/computer.png",\n')
        self.html_file.write('    iconSize:     [32, 32],\n')
        self.html_file.write('    iconAnchor:   [16, 32],\n')
        self.html_file.write('    popupAnchor:  [-3, -33]\n')
        self.html_file.write('});\n')

        self.html_file.write('\n')
        self.html_file.write('var iconQuestion = L.icon({\n')
        self.html_file.write('    iconUrl: "icons/question.png",\n')
        self.html_file.write('    iconSize:     [32, 32],\n')
        self.html_file.write('    iconAnchor:   [16, 32],\n')
        self.html_file.write('    popupAnchor:  [-3, -33]\n')
        self.html_file.write('});\n')


                    
    def genMapHTMLMapTop(self):
        self.html_file.write("\n")
                      
        self.html_file.write("      //basemap.fitBounds(geojson.getBounds());\n")
        self.html_file.write("      map.setView([45.468, -75.553], 15);\n")
        self.html_file.write("      basemap.addTo(map);\n")
        self.html_file.write("\n")
        self.html_file.write("      // add legend control layers - global variable with (null, null) allows indiv basemaps and overlays to be added inside functions below\n")
        self.html_file.write("      var controlLayers = L.control.layers(null, null, {\n")
        self.html_file.write('        position: "topleft",\n')
        self.html_file.write("        collapsed: false // false = open by default\n")
        self.html_file.write("      }).addTo(map);\n")
        self.html_file.write("\n")
        #writeLines("      var markers = [];", self$outputFileHtmlCon)
                      

    def genHTMLBaseMap(self):
        self.html_file.write('  window.onload = function() {\n')
        self.html_file.write("    var basemap = L.tileLayer('https://{s}.tile.osm.org/{z}/{x}/{y}.png', {\n")
        self.html_file.write("      attribution: '&copy; "+'<a href="https://osm.org/copyright">OpenStreetMap</a>'+"'\n")
        self.html_file.write("    });\n")

    def genMarker(self,marker_lat, marker_long, marker_type, marker_name, marker_desc):
        type_data_row = self.type_data[marker_type.lower()]
        #add more data to the json for the marker
        type_data_row = type_data_row[:-1]+',"name":"'+marker_name+'","desc":"'+marker_desc+'"}'
        type_data_row_json = json.loads(type_data_row)  
        self.html_file.write('  markers.push(L.marker(['+str(marker_lat)+','+str(marker_long)+'], {icon: iconQuestion}).addTo(map));\n')
        self.html_file.write('     markers['+str(self.markers)+"].bindPopup('Approach the landmark to see what it is.').openPopup();\n")
        self.html_file.write('     markers_data.push('+type_data_row+')\n')
        self.markers = self.markers +1


    def genPolygon(self,polygon_coods_json, polygon_type):
        # Thanks to: https://gis.stackexchange.com/questions/290609/entering-manual-coordinates-to-draw-polygon-in-leaflet
        fill_color="black"
        type_data_row = self.type_data[polygon_type.lower()]
        type_data_row_json = json.loads(type_data_row)

        fill_color = type_data_row_json['color']
        #if(polygon_type.lower()=="hill"):
        #    fill_color="DarkGreen"
        #if(tolower(polygon_type)=="forest"){fill_color="Green"}
        #if(tolower(polygon_type)=="field"){fill_color="Wheat"}
        #if(tolower(polygon_type)=="road"){fill_color="DarkGrey"}
        #if(tolower(polygon_type)=="creek"){fill_color="Blue"}
        
        self.html_file.write('var coords = '+polygon_coods_json+';\n')
                      #writeLines(paste0('var a = JSON.parse(coords);\n')
        self.html_file.write('polygons.push(L.polygon(coords, {\n')
        self.html_file.write('  color: "white",\n')
        self.html_file.write('  fillColor: "'+fill_color+'",\n')
        self.html_file.write('  weight: 0,\n')
        self.html_file.write('  opacity: 0,\n')
        self.html_file.write('  fillOpacity: 0\n')
        self.html_file.write('}).addTo(map));\n')
        self.html_file.write('     polygons['+str(self.polygons)+"].bindPopup('Approach the landmark to see what it is.').openPopup();\n")
        self.html_file.write('     polygons_data.push('+type_data_row+')\n')
        self.html_file.write('\n')
        self.polygons = self.polygons +1


    def genPolyline(self,p1_lat, p1_long, p2_lat, p2_long):
        self.polylines = self.polylines +1
        self.html_file.write('  var pointA = new L.LatLng(',paste0(p1_lat),',',paste0(p1_long),' );\n')
        self.html_file.write('  var pointB = new L.LatLng(',paste0(p2_lat),',',paste0(p2_long),');\n')
        self.html_file.write('  var pointList = [pointA, pointB];\n')
        self.html_file.write('  \n')
        self.html_file.write('  var polyline'+str(self.polylines)+' = new L.Polyline(pointList, {\n')
        self.html_file.write("      color: 'red',\n")
        self.html_file.write('      weight: 3,\n')
        self.html_file.write('      opacity: 0.5,\n')
        self.html_file.write('      smoothFactor: 1\n')
        self.html_file.write('  });\n')
        self.html_file.write('  polyline'+str(self.polylines)+'.addTo(map);\n')

    def genTestMarker(self,marker_lat, marker_long,):
        self.html_file.write("// To simulate walking around this is a draggable marker. This will greatly reduce testing time.\n")
        self.html_file.write("// Thanks to\n")
        self.html_file.write("// https://stackoverflow.com/questions/57384822/leaflet-detect-when-marker-goes-into-and-out-of-a-circle\n")
        self.html_file.write("var test_marker = new L.marker(["+str(marker_lat)+","+str(marker_long)+"],{\n")
 
        self.html_file.write("    draggable: true,\n")
        self.html_file.write("    autoPan: true\n")
        self.html_file.write("}).addTo(map);\n")
        self.html_file.write("test_marker.setIcon(iconComputer)\n")

        self.html_file.write("test_marker.on('dragend', function (e) {\n")
        self.html_file.write("    document.getElementById('latitude').innerHTML = test_marker.getLatLng().lat;\n")
        self.html_file.write("    document.getElementById('longitude').innerHTML = test_marker.getLatLng().lng;\n")
        self.html_file.write("    document.getElementById('description').innerHTML = distance(test_marker.getLatLng().lat,test_marker.getLatLng().lng,markers[13].getLatLng().lat,markers[13].getLatLng().lng);\n")

        self.html_file.write("    update_per_proximity(test_marker)\n")      
                    

    def genHTMLEndMap(self):
        self.html_file.write("      controlLayers.addOverlay(basemap, 'OpenStreetMap');\n")
        self.html_file.write("      \n")
        self.html_file.write("   // });\n")




        self.html_file.write("});\n")

        self.html_file.write("  };\n")


        self.html_file.write("   // https://gis.stackexchange.com/questions/182068/getting-current-user-location-automatically-every-x-seconds-to-put-on-leaflet\n")
        self.html_file.write("var current_position, current_accuracy;\n")
        self.html_file.write("function onLocationFound(e) {\n")
        self.html_file.write("    // if position defined, then remove the existing position marker and accuracy circle from the map\n")
        self.html_file.write("    if (current_position) {\n")
        self.html_file.write("        map.removeLayer(current_position);\n")
        #writeLines(paste0("       map.removeLayer(current_accuracy);\n")
        self.html_file.write("    }\n")
        self.html_file.write("    var radius = e.accuracy / 2;\n")
        self.html_file.write("    current_position = L.marker(e.latlng).addTo(map)\n")
        self.html_file.write('        .bindPopup("You are within " + radius + " meters from this point");\n')
        self.html_file.write("    current_position.setIcon(iconTravel)\n")
        #writeLines(paste0("current_accuracy = L.circle(e.latlng, radius).addTo(map);\n")
        self.html_file.write("    update_per_proximity(current_position);\n")
        self.html_file.write("}\n")
        self.html_file.write("function onLocationError(e) {\n")
        self.html_file.write("    alert(e.message);\n")
        self.html_file.write("}\n")
        self.html_file.write("map.on('locationfound', onLocationFound);\n")
        self.html_file.write("map.on('locationerror', onLocationError);\n")
        self.html_file.write("// wrap map.locate in a function    \n")
        self.html_file.write("function locate() {\n")
        self.html_file.write("    map.locate({setView: true, maxZoom: 16});\n")
        self.html_file.write("}\n")

        self.html_file.write("// call locate every 3 seconds... forever\n")
        self.html_file.write("setInterval(locate, 3000);\n")

        self.html_file.write("</script>\n")
                    
    def genHTMLEndBody(self):
        self.html_file.write("<hr>\n")
        #self.html_file.write("<p>Page last modifed: ", format(Sys.Date(),"%d %B, %Y"),"</p>\n")
        self.html_file.write("</body>\n")
        self.html_file.write("</html>\n")

    def genLayer(self,geojsonfile):
        self.layers = self.layers +1
        #print(paste0("To map for ", railYear, ", adding: "))
        #print(output_dbRows_route_style[i_route_style, 1])
        layerName = "layer" + str(self.layers)
                      
        self.html_file.write("\n")
        self.html_file.write("// layer for "+geojsonfile+"\n")
        self.html_file.write("var ",layerName,";\n")
                      
        self.html_file.write("      $.ajax({\n")
        self.html_file.write("        type: 'POST',\n")
        self.html_file.write("        url: '",geojsonfile,"',\n")

        self.html_file.write("        dataType: 'json',\n")
        self.html_file.write("        success: function(response) {\n")
        self.html_file.write("          ",layerName," = L.geoJson(response, {\n")
                      
        self.html_file.write("             onEachFeature: function (feature, layer) {\n")

        self.html_file.write("                 layer.bindPopup('<p>'+feature.properties.RTE_TYPE+'</p><p>'+feature.properties.MODE+'</p>');\n")  
                      
        self.html_file.write("               }\n")
        self.html_file.write("             });\n")
        self.html_file.write("          ",layerName,".setStyle({\n")
        self.html_file.write("            color: 'purple',\n")
        self.html_file.write("            weight: 2,\n")
        self.html_file.write('            opacity: 1\n')
        self.html_file.write("          });\n")
        self.html_file.write("          ",layerName,".addTo(map);\n")
        self.html_file.write("          controlLayers.addOverlay(",layerName,", '<span style=",'"color: purple"',">route</span>');\n")

        self.html_file.write("        }\n")
        self.html_file.write("      });\n")
        self.html_file.write("\n")
                      

gm = GenerateLeafletMap("/content/drive/MyDrive/github/sensation/userloc.html")
gm.genMapHTMLTop()


type_data_dict={
    "hill":'{"type":"hill","color":"brown","sight":"hilly terrain","hearing":"wind blowing through trees","smell":"damp leaves","feeling":"cool, damp","sound":""}',
    "rch":'{"type":"rch","color":"","sight":"a flat circle","hearing":"wind blowing through trees","smell":"ash","feeling":"cool, damp","sound":"rch.wav"}',
    "forest":'{"type":"forest","color":"green","sight":"level terrain, trees","hearing":"wind blowing through trees","smell":"damp leaves","feeling":"cool, damp","sound":""}',
    "field":'{"type":"field","color":"wheat","sight":"level terrain, grass","hearing":"wind blowing through grass","smell":"grass","feeling":"warm, sun","sound":""}',
    "road":'{"type":"road","color":"gray","sight":"a muddy road","hearing":"squash of footsteps","smell":"mud","feeling":"sun","sound":"road.wav"}',
    "path":'{"type":"path","color":"gray","sight":"a muddy path","hearing":"squash of footsteps","smell":"mud","feeling":"sun","sound":"path.wav"}',
    "creek":'{"type":"creek","color":"blue","sight":"a running creek","hearing":"water rippling","smell":"fresh water","feeling":"sun","sound":"creek.wav"}',
    "business":'{"type":"business","color":"","sight":"a building","hearing":"horses","smell":"smoke","feeling":"cool, damp","sound":""}',
    "furnace":'{"type":"furnace","color":"","sight":"hilly terrain","hearing":"wind blowing through trees","smell":"damp leaves","feeling":"cool, damp","sound":"furnace.wav"}',
    "house":'{"type":"house","color":"","sight":"a house","hearing":"people talking","smell":"wood smoke","feeling":"cool, damp","sound":""}'
}
gm.set_type_data_dict(type_data_dict)

gm.genMapHTMLH1("Hopewell, Pennsylvania")
gm.genMapHTMLScriptTop()
gm.genHTMLBaseMap()
gm.genMapHTMLMapTop()

#--------------------------------------------------------
# Paste below
#--------------------------------------------------------

gm.genMarker(45.48069524143359, -75.55294256515234,"RCH","RCH","")
gm.genMarker(45.479835774616625, -75.54924580025349,"RCH","","")
gm.genMarker(45.48087461798589, -75.54745352258493,"RCH","","")
gm.genMarker(45.48159160526861, -75.54374033081376,"RCH","","")
gm.genMarker(45.48074292337904, -75.54225039323578,"RCH","","")
gm.genMarker(45.481173812195195, -75.54037418908419,"RCH","","")
gm.genMarker(45.481570547824525, -75.53939571220832,"RCH","","")
gm.genMarker(45.47208041856893, -75.53911713843256,"RCH","","")
gm.genMarker(45.47212550944454, -75.53739164423271,"RCH","","")
gm.genMarker(45.47153426016818, -75.53834179690928,"RCH","","")
gm.genMarker(45.47052250587142, -75.53878836432347,"RCH","","")
gm.genMarker(45.47107363424301, -75.53987385471375,"RCH","","")
gm.genMarker(45.46921990642112, -75.54531261117356,"RCH","","")
gm.genMarker(45.47816607361546, -75.54191997834484,"Furnace","Clingan & Buckley's Furnace","Clingan & Buckley's Furnace")
gm.genMarker(45.477661000000005, -75.536143,"Business","Clingan & Buckley","Clingan & Buckley")
gm.genMarker(45.47630702214738, -75.53642835143401,"House","","")
gm.genMarker(45.475436884379604, -75.53662901664491,"House","","")
gm.genMarker(45.47467801918609, -75.53620538296144,"House","","")
gm.genMarker(45.474633279317196, -75.53493099286987,"House","","S.H. Is this a school house?")
gm.genMarker(45.473971523461664, -75.53602143272323,"House","Clingan & Buckley","Clingan & Buckley's house.")
gm.genMarker(45.47307535351896, -75.53460232148552,"House","N. Care","N. Care's house.")
gm.genMarker(45.47537087556559, -75.53856218959736,"House","","")
gm.genMarker(45.47484413276825, -75.54051028187385,"House","","")
gm.genMarker(45.4742765520621, -75.54142519878856,"House","","")
gm.genPolygon('[[45.47241415488257,-75.53572719742755],[45.47238935371979,-75.53569352210094],[45.47238935371979,-75.53569352210094],[45.47238935371979,-75.53569352210094],[45.47234085901102,-75.53569509718041],[45.47234085901102,-75.53569509718041],[45.47197604106905,-75.53563798499944],[45.46805269997154,-75.5421093780862],[45.4682206938845,-75.54958569802682],[45.46955257782138,-75.55095641199402],[45.46987837516826,-75.54856686042025],[45.47318486702365,-75.54142606215255],[45.47380079101707,-75.537475208911],[45.47241415488257,-75.53572719742755]]','hill')
gm.genPolygon('[[45.47214078761794,-75.53532230780189],[45.47221352967083,-75.53531994469992],[45.47218928231977,-75.53532073240127],[45.47218928231977,-75.53532073240127],[45.47218928231977,-75.53532073240127],[45.47729895024767,-75.53625820894537],[45.47734412298491,-75.53604983855001],[45.47218706652228,-75.53518288151251],[45.47214078761794,-75.53532230780189]]','road')
gm.genPolygon('[[45.47580823176229,-75.53558247203736],[45.47704970354474,-75.53433521254571],[45.47693552785559,-75.53326992955068],[45.47687354597413,-75.53092705595019],[45.47634435501526,-75.52970288970072],[45.47572205340078,-75.52872316386939],[45.47233144265936,-75.53510922931032],[45.47580823176229,-75.53558247203736]]','field')
gm.genPolygon('[[45.47650015207343,-75.55611204320581],[45.4759563020478,-75.5554743330175],[45.47578835159154,-75.55406593468511],[45.47513817652603,-75.55284547143914],[45.47412043355972,-75.55139549698119],[45.4742395551631,-75.54973653313603],[45.47372876855899,-75.54813236136216],[45.47402527971506,-75.54695041807496],[45.47385556963202,-75.54543870321007],[45.47430194596157,-75.54452775292533],[45.474693202849366,-75.54320478374017],[45.47563385469154,-75.54134676971047],[45.475564925972925,-75.54007313910543],[45.47581064597427,-75.53875482570241],[45.47610429194899,-75.53740046174431],[45.47585628474507,-75.53706367986828],[45.47563538502541,-75.53841568036472],[45.47543208491983,-75.5393533068027],[45.475231538822236,-75.54046324969228],[45.474513542252495,-75.54258991717792],[45.474192829687574,-75.54377267829486],[45.47370788174085,-75.54530554785137],[45.47365276130221,-75.54641073741054],[45.47348959380013,-75.54834696436754],[45.473952431806076,-75.54998715778059],[45.47392864962624,-75.5515395982866],[45.474872002733555,-75.55288851663674],[45.47552765818474,-75.55445362442379],[45.47574026529492,-75.55561920381696],[45.47623835671419,-75.55643079737537],[45.47650015207343,-75.55611204320581]]','creek')
gm.genPolygon('[[45.480328664562556,-75.55429929748178],[45.48416668061128,-75.53941501180533],[45.48214861697988,-75.53913556406698],[45.478760533807375,-75.5381418956489],[45.47846699251775,-75.54404827994126],[45.479112290192724,-75.54647584644002],[45.47825694032927,-75.54912427642176],[45.47884814306308,-75.55120878393257],[45.479822331144774,-75.55297064818701],[45.480328664562556,-75.55429929748178]]','forest')
gm.genPolygon('[[45.476370215762586,-75.55762776215293],[45.47661248866527,-75.55581382275828],[45.47633254935776,-75.55312561776097],[45.476197276713286,-75.54995113590451],[45.47658145694551,-75.54748236601007],[45.477417401751225,-75.54572144113446],[45.478299503606614,-75.54472962681909],[45.4781751941827,-75.54437239974185],[45.47729348019775,-75.54538828768578],[45.47642405737641,-75.54717437650741],[45.47590633232837,-75.54976785554662],[45.47613815159834,-75.5536857575111],[45.476291128329414,-75.55584822621424],[45.47615936767907,-75.55715288695157],[45.476370215762586,-75.55762776215293]]','path')
gm.genPolygon('[[45.47027322025065,-75.5545726360258],[45.46997356180865,-75.55277631517451],[45.4699786826861,-75.5509701984859],[45.47033823211261,-75.54908041705121],[45.47144900692648,-75.54649212630326],[45.47324553498704,-75.54335177984477],[45.47386821358054,-75.54203126102894],[45.47487799566366,-75.53949409495162],[45.47547326499539,-75.53752417141177],[45.475974321579486,-75.53707443564912],[45.47619593087976,-75.5361039560616],[45.476008499729915,-75.5360377964398],[45.475852693473016,-75.5368857271083],[45.475268900973305,-75.53745855732818],[45.47400866729891,-75.54128019841659],[45.47137704262774,-75.54622956949646],[45.47016582305329,-75.54889333914808],[45.46977471027899,-75.5509286081749],[45.46980652233358,-75.55292616432985],[45.47003576428557,-75.55455618769277],[45.47027322025065,-75.5545726360258]]','road')
gm.genPolygon('[[45.48062034583925,-75.55640760822135],[45.4798058980805,-75.55313430375257],[45.47871262701813,-75.55150773375172],[45.478183951862285,-75.54916462704828],[45.4789333537904,-75.54622638895411],[45.478288341514364,-75.54403157973333],[45.4785825711235,-75.53807352113051],[45.47839629984562,-75.53807956374696],[45.478139786973365,-75.5442772124494],[45.478744390437306,-75.54606390856583],[45.477978442905425,-75.5490267552862],[45.47861447420303,-75.5517276420633],[45.479637708825294,-75.55321196475754],[45.48043598502703,-75.55653394733872],[45.48062034583925,-75.55640760822135]]','road')
gm.genPolygon('[[45.482761406553024,-75.53875684022476],[45.482756000524724,-75.53841982533766],[45.48153831402938,-75.53855565946148],[45.48005889777579,-75.53821829859424],[45.47850904169509,-75.53771465886831],[45.47832347367183,-75.53670918991577],[45.47750794801377,-75.5365670859692],[45.47744902746776,-75.5360632618048],[45.477295463727515,-75.53599600091488],[45.47732515585417,-75.53678976455966],[45.4781402946958,-75.5369078008496],[45.47842939482964,-75.53803032388409],[45.479943451096,-75.53841471380555],[45.48166186993225,-75.53886475115229],[45.482761406553024,-75.53875684022476]]','road')
gm.genPolygon('[[45.47827151276462,-75.53663862757395],[45.47820099001147,-75.53541268164304],[45.47818435160209,-75.53437765105147],[45.47801501426572,-75.5343831556167],[45.47810480266026,-75.53575296870598],[45.47813604280658,-75.5366430256981],[45.47827151276462,-75.53663862757395]]','road')
gm.genPolygon('[[45.47866087506933,-75.54613885527307],[45.47832067014963,-75.54499385616441],[45.47747012610219,-75.54584015139987],[45.47673501374601,-75.54754965792077],[45.47628271311651,-75.5499965470521],[45.4767618334406,-75.55561636781123],[45.48031706581726,-75.55651368165591],[45.47955495461945,-75.5533350442814],[45.47844705255589,-75.55185344708758],[45.47781217590841,-75.54922477827711],[45.47866087506933,-75.54613885527307]]','field')
gm.genPolygon('[[45.47618302324308,-75.55544230365277],[45.47580664816963,-75.54989147760553],[45.476271653276214,-75.54717929520119],[45.47723806228132,-75.54510108727924],[45.47803741443597,-75.5442323560515],[45.478310858566196,-75.53803416927235],[45.47802214497981,-75.53693571904228],[45.477343635685486,-75.53688549533895],[45.47719966124973,-75.53636035144929],[45.47624866542451,-75.536222654537],[45.47609208480379,-75.53702244920652],[45.47571662499949,-75.5400207974247],[45.475708596986614,-75.54163454689619],[45.47454443751278,-75.54405629402864],[45.474042964038816,-75.54554145806695],[45.474067186755335,-75.5470577898187],[45.47386433690062,-75.5481479853645],[45.47436305428514,-75.54967310154387],[45.47420248290076,-75.55129171729344],[45.47585295683311,-75.55386350155958],[45.47602755847257,-75.55525464210669],[45.47618302324308,-75.55544230365277]]','field')
gm.genPolygon('[[45.47572111401256,-75.55622776071374],[45.475424543655734,-75.5546238092559],[45.47484084158312,-75.55307727349552],[45.47374757065075,-75.55145085062257],[45.47380999489017,-75.55005214083054],[45.47336016396529,-75.5484050558907],[45.473661187984135,-75.54497584601971],[45.4751955367672,-75.54027849925704],[45.47564376150438,-75.53759088487938],[45.47544673769539,-75.53798258709159],[45.474102202753585,-75.54183103799477],[45.470425204256266,-75.54922209083729],[45.47011568687517,-75.55106210440123],[45.47009171527541,-75.55274843487626],[45.47042677259787,-75.55463993994407],[45.47572111401256,-75.55622776071374]]','field')
gm.genPolygon('[[45.47577374750997,-75.5361899120091],[45.472647886784706,-75.5356653141525],[45.473913815617294,-75.53747844649862],[45.47328668428221,-75.54168886938163],[45.473413314495176,-75.54219046955245],[45.47521578133174,-75.53731579057639],[45.47573415730554,-75.5368895747704],[45.47577374750997,-75.5361899120091]]','field')
gm.genTestMarker(45.47765813606775, -75.56172205625913)

#--------------------------------------------------------
# Paste above
#--------------------------------------------------------

gm.genHTMLEndMap()
gm.genHTMLEndBody()

gm.close()

# 4. Test generated map using http server on Google Colab

To test the map, we can use an http server on Google Colab. 
* Run the cell below to generate the URL for the test map.
* Start the http server
* Click the URL to launch the map in a web browser

Once your map is ready, the userloc.html file can be placed on an http server that has TLS/https and accessed from a smartphone.

In [ ]:
# Start the server, then click on the URL
from google.colab.output import eval_js
print("Click this URL when the server starts.")
print(eval_js("google.colab.kernel.proxyPort(8000)")+"drive/MyDrive/github/sensation/userloc.html")
!python -m http.server 8000